# Tests and implementation for the RFClassifier class.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as la
from estimator import RFClassifier, classical_weights, V1_inspired_weights, haltere_inspired_weights

## Implementation for V1 weights

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
data, labels = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

b = np.mean(la.norm(X_train, axis=1) ** 2 / X_train.shape[1])

In [ ]:
# select the linear classifier
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(solver='saga')

# non-linearity
relu = lambda x: np.maximum(0, x)

# classify using classical weights
params_classical = {'weight_fun': classical_weights, 'bias': b, 'nonlinearity': relu, 'clf': logit}
clf = RFClassifier(width=40, **params_classical)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

# classify using V1 weights
kwargs = {'t': 5, 'l': 3}
params_neural = {'weight_fun': V1_inspired_weights, 'kwargs': kwargs, 'bias': b, 'nonlinearity':relu, 'clf': logit}
clf = RFClassifier(width=40, **params_neural)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

## Implementation for haltere weights

In [ ]:

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
data, labels = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)
b = np.mean(la.norm(X_train, axis=1) ** 2 / X_train.shape[1])

In [ ]:
# linear classifier
logit = LogisticRegression(solver='saga')

# non-linearity
cos = lambda x: np.cos(x)
relu = lambda x: np.maximum(x, 0)

# params
params_classical = {'weight_fun': classical_weights, 'bias': b, 'nonlinearity': relu, 'clf': logit}
clf = RFClassifier(60, **params_classical)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
W_classical = clf.W_[0]

kwargs = {'lowcut': 4, 'highcut': 6}
params_neural = {'weight_fun': haltere_inspired_weights, 'kwargs': kwargs, 'bias': b, 'nonlinearity': relu, 'clf': logit}
clf = RFClassifier(60, **params_neural)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))
W_neural = clf.W_[0]

fig = plt.figure()
plt.plot(W_classical, c='k')
plt.plot(W_neural)

### Time the linearSVM vs logistic regression

In [3]:
from estimator import RFClassifier, classical_weights, relu, haltere_inspired_weights, V1_inspired_weights
from data_fns import frequency_XOR, load_mnist
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import numpy as np
import numpy.linalg as la
import time

In [4]:
# X, y = frequency_XOR(60000, 1000, 5, 8, 0.784, 0.99, random_state=5, shuffle=True)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=5)
# b = np.mean(la.norm(X_train, axis=1) ** 2 / X_train.shape[1])
X_train, y_train, X_test, y_test = load_mnist('data/mnist/')
b = np.mean(la.norm(X_train, axis=1) ** 2 / X_train.shape[1])

In [4]:
# start = time.time()
# # logistic regression
# logit = LogisticRegression(dual=False, solver='saga', n_jobs=5)
# params = {'width': 20, 'weight_fun': classical_weights, 'bias': b, 'nonlinearity': relu, 'clf': logit}
# clf = RFClassifier(**params)
# clf.fit(X_train, y_train)
# print(clf.score(X_test, y_test))
# end = time.time()
# print('%0.3f s' % (end - start))

0.785
3.062 s


In [14]:
time_taken = []
for i in range(2):
    start = time.time()
    svm = LinearSVC(dual=True, max_iter=200)
    kwargs = {'t': 5, 'l':3}
    params = {'width': 1000, 'weight_fun': V1_inspired_weights, 'kwargs': kwargs, 'bias': b, 'nonlinearity': relu, 'clf': svm}
    clf = RFClassifier(**params)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    end = time.time()
    print('Iter %d: score=%0.3f, %0.3f s' % (i, score, end - start))
    time_taken.append(end - start)
print(np.mean(time_taken), np.std(time_taken))

Iter 0: score=0.978, 121.153 s
Iter 1: score=0.975, 102.257 s
111.7051830291748 9.447813034057617


In [15]:
time_taken = []
for i in range(2):
    start = time.time()
    svm = LinearSVC(dual=False, max_iter=200)
    kwargs = {'t': 5, 'l':3}
    params = {'width': 1000, 'weight_fun': V1_inspired_weights, 'kwargs': kwargs, 'bias': b, 'nonlinearity': relu, 'clf': svm}
    clf = RFClassifier(**params)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    end = time.time()
    print('Iter %d: score=%0.3f, %0.3f s' % (i, score, end - start))
    time_taken.append(end - start)
print(np.mean(time_taken), np.std(time_taken))

Iter 0: score=0.980, 150.549 s
Iter 1: score=0.978, 158.188 s
154.36838352680206 3.819311261177063


In [17]:
from sklearn.linear_model import SGDClassifier
time_taken = []
for i in range(2):
    start = time.time()
    sgd = SGDClassifier(loss='squared_hinge', alpha=1, shuffle=True, n_jobs=5, learning_rate='optimal', max_iter=500)
    kwargs = {'t': 5, 'l':3}
    params = {'width': 1000, 'weight_fun': V1_inspired_weights, 'kwargs': kwargs, 'bias': b, 
'nonlinearity': relu, 'clf': sgd}
    clf = RFClassifier(**params)
    clf.fit(X_train, y_train)

    score = clf.score(X_test, y_test)
    end = time.time()
    print('Iter %d: score=%0.3f, %0.3f s' % (i, 1 - score, end - start))
    time_taken.append(end - start)
print(np.mean(time_taken), np.std(time_taken))

Iter 0: score=0.049, 100.192 s
Iter 1: score=0.041, 118.715 s
109.45346260070801 9.261189460754395


316